In [2]:
import pandas as pd
import re

In [3]:
df = pd.read_csv('/Users/shubham.agnihotri/Documents/GitHub/Law-RAG/data/justice.csv')
df.head()

,Unnamed: 0,ID,name,href,docket,term,first_party,second_party,facts,facts_len,majority_vote,minority_vote,first_party_winner,decision_type,disposition,issue_area
0,0,50606,Roe v. Wade,https://api.oyez.org/cases/1971/70-18,70-18,1971,Jane Roe,Henry Wade,"<p>In 1970, Jane Roe (a fictional name used in...",501,7,2,True,majority opinion,reversed,NaN
1,1,50613,Stanley v. Illinois,https://api.oyez.org/cases/1971/70-5014,70-5014,1971,"Peter Stanley, Sr.",Illinois,<p>Joan Stanley had three children with Peter ...,757,5,2,True,majority opinion,reversed/remanded,Civil Rights
2,2,50623,Giglio v. United States,https://api.oyez.org/cases/1971/70-29,70-29,1971,John Giglio,United States,<p>John Giglio was convicted of passing forged...,495,7,0,True,majority opinion,reversed/remanded,Due Process
3,3,50632,Reed v. Reed,https://api.oyez.org/cases/1971/70-4,70-4,1971,Sally Reed,Cecil Reed,"<p>The Idaho Probate Code specified that ""male...",378,7,0,True,majority opinion,reversed/remanded,Civil Rights
4,4,50643,Miller v. California,https://api.oyez.org/cases/1971/70-73,70-73,1971,Marvin Miller,California,"<p>Miller, after conducting a mass mailing cam...",305,5,4,True,majority opinion,vacated/remanded,First Amendment


In [4]:
df.columns

Index(['Unnamed: 0', 'ID', 'name', 'href', 'docket', 'term', 'first_party',
       'second_party', 'facts', 'facts_len', 'majority_vote', 'minority_vote',
       'first_party_winner', 'decision_type', 'disposition', 'issue_area'],
      dtype='object')

In [5]:
# df = df.drop(['Unnamed: 0',])
data_df = df[['first_party','second_party','facts','first_party_winner','ID']]

In [6]:
data_df

,first_party,second_party,facts,first_party_winner,ID
0,Jane Roe,Henry Wade,"<p>In 1970, Jane Roe (a fictional name used in...",True,50606
1,"Peter Stanley, Sr.",Illinois,<p>Joan Stanley had three children with Peter ...,True,50613
2,John Giglio,United States,<p>John Giglio was convicted of passing forged...,True,50623
3,Sally Reed,Cecil Reed,"<p>The Idaho Probate Code specified that ""male...",True,50632
4,Marvin Miller,California,"<p>Miller, after conducting a mass mailing cam...",True,50643
...,...,...,...,...,...
3298,United States,Refugio Palomar-Santiago,"<p>Refugio Palomar-Santiago, a Mexican nationa...",True,63324
3299,Tarahrick Terry,United States,<p>Tarahrick Terry pleaded guilty to one count...,False,63323
3300,United States,Joshua James Cooley,<p>Joshua James Cooley was parked in his picku...,True,63331
3301,Florida,Georgia,<p>This is an ongoing case of original jurisdi...,False,63332


In [7]:
def clean_text(text):

    # Remove HTML tags (e.g., <p>, </p>, <strong>, <em>, etc.)
    text = re.sub(r'<[^>]+>', '', text)

    # Remove special characters using a regular expression
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)
    return text

In [8]:
data_df['facts'] = data_df['facts'].apply(clean_text)
data_df = data_df[data_df['first_party_winner'] != '']
data_df

/var/folders/sv/tpnlt64j4vngl93f1r3z4vj40000gp/T/ipykernel_42893/399892728.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df['facts'] = data_df['facts'].apply(clean_text)


,first_party,second_party,facts,first_party_winner,ID
0,Jane Roe,Henry Wade,In 1970 Jane Roe a fictional name used in cour...,True,50606
1,"Peter Stanley, Sr.",Illinois,Joan Stanley had three children with Peter Sta...,True,50613
2,John Giglio,United States,John Giglio was convicted of passing forged mo...,True,50623
3,Sally Reed,Cecil Reed,The Idaho Probate Code specified that males mu...,True,50632
4,Marvin Miller,California,Miller after conducting a mass mailing campaig...,True,50643
...,...,...,...,...,...
3298,United States,Refugio Palomar-Santiago,Refugio PalomarSantiago a Mexican national was...,True,63324
3299,Tarahrick Terry,United States,Tarahrick Terry pleaded guilty to one count of...,False,63323
3300,United States,Joshua James Cooley,Joshua James Cooley was parked in his pickup t...,True,63331
3301,Florida,Georgia,This is an ongoing case of original jurisdicti...,False,63332


In [9]:
meta_df = df.drop(['facts','first_party_winner','facts_len','Unnamed: 0'],axis = 1)
meta_df

,ID,name,href,docket,term,first_party,second_party,majority_vote,minority_vote,decision_type,disposition,issue_area
0,50606,Roe v. Wade,https://api.oyez.org/cases/1971/70-18,70-18,1971,Jane Roe,Henry Wade,7,2,majority opinion,reversed,NaN
1,50613,Stanley v. Illinois,https://api.oyez.org/cases/1971/70-5014,70-5014,1971,"Peter Stanley, Sr.",Illinois,5,2,majority opinion,reversed/remanded,Civil Rights
2,50623,Giglio v. United States,https://api.oyez.org/cases/1971/70-29,70-29,1971,John Giglio,United States,7,0,majority opinion,reversed/remanded,Due Process
3,50632,Reed v. Reed,https://api.oyez.org/cases/1971/70-4,70-4,1971,Sally Reed,Cecil Reed,7,0,majority opinion,reversed/remanded,Civil Rights
4,50643,Miller v. California,https://api.oyez.org/cases/1971/70-73,70-73,1971,Marvin Miller,California,5,4,majority opinion,vacated/remanded,First Amendment
...,...,...,...,...,...,...,...,...,...,...,...,...
3298,63324,United States v. Palomar-Santiago,https://api.oyez.org/cases/2020/20-437,20-437,2020,United States,Refugio Palomar-Santiago,9,0,majority opinion,reversed/remanded,Criminal Procedure
3299,63323,Terry v. United States,https://api.oyez.org/cases/2020/20-5904,20-5904,2020,Tarahrick Terry,United States,9,0,majority opinion,affirmed,Criminal Procedure
3300,63331,United States v. Cooley,https://api.oyez.org/cases/2020/19-1414,19-1414,2020,United States,Joshua James Cooley,9,0,majority opinion,vacated/remanded,Civil Rights
3301,63332,Florida v. Georgia,https://api.oyez.org/cases/2020/142-orig,142-orig,2020,Florida,Georgia,9,0,majority opinion,none,NaN


In [10]:
meta_df.to_csv('meta_df.csv', index = False)

In [12]:
data_df.to_csv('data.csv', index = False)